# OLS를 이용해 다음 문제를 풀어봅시다.

## Q1.

삼성전자 주식 데이터와 코스피 데이터를 불러와서 beta를 구한 뒤, 삼성에 대한 요구수익률을 CAPM을 통해 계산해 봅시다. 단, 무위험이자율은 https://www.kofr.kr/main.jsp 에서 제시하는 3.499%를 사용하겠습니다.


In [51]:
# 선형회귀분석은 다음과 같은 코드를 이용합니다.
import statsmodels.api as sm
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import pandas as pd
import pandas_datareader as pdr
import numpy as np

r_f = 0.03499

start_date = pd.to_datetime('2015-01-01')
end_date = pd.to_datetime('2024-10-04')
samsung= pdr.DataReader("005930", "naver", start_date, end_date).astype(int)
kospi = pdr.DataReader('KOSPI', "naver", start_date, end_date).astype(float)
monthly_sam = samsung.resample('M').last()
monthly_kos = kospi.resample('M').last()
ret_sam = np.log(monthly_sam.Close) - np.log(monthly_sam.Close.shift(1))
ret_kospi = np.log(monthly_kos.Close) - np.log(monthly_kos.Close.shift(1))

df=pd.concat([ret_sam, ret_kospi], axis=1).dropna()
df.columns=['sam', 'KOSPI']

X = df.KOSPI-r_f
X = sm.add_constant(X)
y = df.sam-r_f

model = sm.OLS(y, X)
result = model.fit()

beta=result.params[1]
sam_exret=r_f+beta*(np.mean(ret_kospi*12)-r_f)
print(beta, sam_exret)

1.0970571136499652 0.027697747575491354


C:\Users\kumb2\AppData\Local\Temp\ipykernel_37404\3673617232.py:30: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  beta=result.params[1]


## Q2.

선형대수학 꼴로 나타낸 OLS 문제는 다음과 같습니다

$$y = X \beta + u$$

$\beta$를 구하기 위해 우리는 다음 목적함수를 최소화하는 beta를 계산하고 싶습니다.

$$\min_{\hat{\beta}} \hat{u}^{T}\hat{u}$$

이를 위 회귀식을 이용해 표현하면 목적함수는 다음과 같이 변경됩니다.

$$\min_{\hat{\beta}}(Y-X\hat{\beta})^{T}(Y-X\hat{\beta})$$

이를 풀어보면 $\hat{\beta}$은 다음과 같이 계산됩니다.  

$$\hat{\beta} = (X^TX)^{-1}X^{T} y $$

- 다음 데이터를 OLS를 이용해 푼 결과와 numpy를 이용해 계산해 본 결과와 비교해보세요

In [75]:
df1 = pd.read_stata('https://github.com/QuantEcon/lecture-python/blob/master/source/_static/lecture_specific/ols/maketable1.dta?raw=true')
df1 = df1.dropna(subset=['logpgp95', 'avexpr'])

df1['const'] = 1

reg1 = sm.OLS(endog=df1['logpgp95'], exog=df1[['const', 'avexpr']], missing='drop')
res = reg1.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:               logpgp95   R-squared:                       0.611
Model:                            OLS   Adj. R-squared:                  0.608
Method:                 Least Squares   F-statistic:                     171.4
Date:                Fri, 04 Oct 2024   Prob (F-statistic):           4.16e-24
Time:                        15:52:24   Log-Likelihood:                -119.71
No. Observations:                 111   AIC:                             243.4
Df Residuals:                     109   BIC:                             248.8
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          4.6261      0.301     15.391      0.0

In [76]:
# Define the X and y variables (Use numpy)
X=np.array(df1[['const', 'avexpr']])
y=np.array(df1['logpgp95'])

# Compute β_hat
β_hat=np.dot( np.dot( np.linalg.inv( np.dot(np.transpose(X), X)), np.transpose(X)), y )

# Print out the results from the 2 x 1 vector β_hat
print(f'β_0 = {β_hat[0]:.2}')
print(f'β_1 = {β_hat[1]:.2}')

β_0 = 4.6
β_1 = 0.53
